In [1]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sktime.classification.kernel_based import RocketClassifier
from sktime.classification.interval_based import CanonicalIntervalForest
from sktime.classification.dictionary_based import MUSE
from sktime.transformations.series.sax import SAX
from sktime.transformations.series.paa import PAA
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics.pairwise import linear_kernel, polynomial_kernel, rbf_kernel
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [2]:
tracks_dataset_grouped_genres = pd.read_csv('dataset/tabular/tracks_dataset_without_outliers.csv')
# conversion to a list
tracks_dataset_grouped_genres['genre'] = tracks_dataset_grouped_genres['genre'].apply(lambda x: x.replace('[','').replace(']','').replace('\'','').split(','))
# keeping only the first genre
tracks_dataset_grouped_genres['genre'] = tracks_dataset_grouped_genres['genre'].apply(lambda x: x[0])

# dropping useless attributes from the dataset
tracks_dataset_grouped_genres = tracks_dataset_grouped_genres.drop(columns=['id', 'name', 'artists', 'album_name', 'n_bars', 'month', 'day'])

# one-hot encode explicit attribute with 0 and 1 in the same column
tracks_dataset_grouped_genres['explicit'] = tracks_dataset_grouped_genres['explicit'].apply(lambda x: 1 if x == True else 0)

# Convert to lowercase for case-insensitive matching
tracks_dataset_grouped_genres['genre'] = tracks_dataset_grouped_genres['genre'].str.lower()

# Grouping similar genres by checking if the keyword is contained in the label
tracks_dataset_grouped_genres['genre'] = tracks_dataset_grouped_genres['genre'].apply(lambda x: 'rock' if any(kw in x for kw in ['rock', 'punk', 'alternative','garage', 'grunge', 'ska']) else x)
tracks_dataset_grouped_genres['genre'] = tracks_dataset_grouped_genres['genre'].apply(lambda x: 'metal' if any(kw in x for kw in ['metal', 'grindcore']) else x)
tracks_dataset_grouped_genres['genre'] = tracks_dataset_grouped_genres['genre'].apply(lambda x: 'pop' if any(kw in x for kw in ['pop', 'r&b', 'soul', 'funk', 'reggae', 'r-n-b']) else x)
tracks_dataset_grouped_genres['genre'] = tracks_dataset_grouped_genres['genre'].apply(lambda x: 'electronic' if any(kw in x for kw in ['electronic', 'house', 'techno', 'trance','electro','dub', 'dubstep', 'industrial', 'drum-and-bass']) else x)
tracks_dataset_grouped_genres['genre'] = tracks_dataset_grouped_genres['genre'].apply(lambda x: 'hip-hop' if any(kw in x for kw in ['hip-hop', 'rap', 'trap', 'grime']) else x)
#tracks_dataset_grouped_genres['genre'] = tracks_dataset_grouped_genres['genre'].apply(lambda x: 'jazz' if any(kw in x for kw in ['jazz', 'blues', 'country', 'folk', 'classical']) else x)
tracks_dataset_grouped_genres['genre'] = tracks_dataset_grouped_genres['genre'].apply(lambda x: 'national' if any(kw in x for kw in ['british', 'swedish', 'spanish', 'brazil', 'indian', 'iranian', 'german', 'french', 'turkish']) else x)

# Convert back to title case for consistency
tracks_dataset_grouped_genres['genre'] = tracks_dataset_grouped_genres['genre'].str.title()

# Print unique genres to verify
print(tracks_dataset_grouped_genres['genre'].unique())

# count records in tracks_dataset_grouped_genres
print(tracks_dataset_grouped_genres['genre'].value_counts())

# remove all records with genre different from electronic, pop, national, rock, metal or jazz
tracks_dataset_grouped_genres = tracks_dataset_grouped_genres[tracks_dataset_grouped_genres['genre'].isin(['Electronic', 'Pop', 'National', 'Rock', 'Metal'])]

print(tracks_dataset_grouped_genres['genre'].value_counts())


['Pop' 'Indie' 'Hip-Hop' 'Songwriter' 'Rock' 'Electronic' 'Country'
 'Dance' 'Metal' 'Latino' 'Folk' 'Latin' 'Piano' 'Emo' 'Chill' 'National'
 'Blues' 'Sad' 'Edm' 'Hardcore' 'Disco' 'Acoustic' 'Dancehall' 'J-Dance'
 'Jazz' 'Ambient' 'Salsa' 'Afrobeat' 'Show-Tunes' 'Groove' 'Anime'
 'Comedy' 'Trip-Hop' 'Children' 'New-Age' 'Disney' 'World-Music' 'Idm'
 'Opera' 'Party' 'Mpb' 'Classical' 'Bluegrass' 'Sleep' 'Happy' 'Samba'
 'Breakbeat' 'Pagode' 'Goth' 'J-Idol' 'Sertanejo' 'Kids' 'Club' 'Malay'
 'Hardstyle' 'Gospel' 'Forro' 'Guitar' 'Honky-Tonk' 'Study' 'Tango'
 'Romance']
genre
Electronic    9811
Pop           9016
National      7493
Rock          7332
Metal         4521
              ... 
Dance          246
Latin          196
Indie          185
Edm            139
Latino         138
Name: count, Length: 62, dtype: int64
genre
Electronic    9811
Pop           9016
National      7493
Rock          7332
Metal         4521
Name: count, dtype: int64


In [3]:

# get the mapping of genres to integers
genres = tracks_dataset_grouped_genres['genre'].unique()
genre_to_int = {genre: i for i, genre in enumerate(genres)}
int_to_genre = {i: genre for i, genre in enumerate(genres)}

# convert genres to integers
tracks_dataset_grouped_genres['genre'] = tracks_dataset_grouped_genres['genre'].apply(lambda x: genre_to_int[x])

# Extract the features and the target where the target is the genre
X = tracks_dataset_grouped_genres.drop(columns=['genre'])
y = tracks_dataset_grouped_genres['genre']

#Extract train and test from the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# normalize the dataset
#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)

In [6]:
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

param_grid = {
    'n_estimators': [100, 300, 500],
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': [None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'min_weight_fraction_leaf': [0.0, 0.1, 0.3, 0.5],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False],
    'random_state': [42],
    'n_jobs': [-1]
}

grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:542: FitFailedWarning: 
1944 fits failed out of a total of 5832.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
714 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 890, in _fit_and

{'bootstrap': False, 'criterion': 'gini', 'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 5, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 500, 'n_jobs': -1, 'random_state': 42}


In [4]:
clf = RandomForestClassifier(n_estimators=500,
                             criterion='gini',
                             max_depth=None,
                             min_samples_split=5,
                             min_samples_leaf=1,
                             min_weight_fraction_leaf=0.0,
                             max_features='log2',
                             oob_score=True,
                             random_state=42,
                             n_jobs=-1)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))
print(clf.oob_score_)

Accuracy 0.6678454485920104
F1-score [0.63684489 0.60631995 0.76881188 0.76735147 0.55951014]
              precision    recall  f1-score   support

           0       0.61      0.67      0.64      1803
           1       0.59      0.63      0.61      1467
           2       0.75      0.79      0.77      1962
           3       0.77      0.76      0.77       904
           4       0.66      0.49      0.56      1499

    accuracy                           0.67      7635
   macro avg       0.67      0.67      0.67      7635
weighted avg       0.67      0.67      0.67      7635

0.6608487785709608


# Explainability

## LORE